In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics

C:\Users\HP\miniconda3\envs\gpu\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
transform= transforms.Compose([transforms.ToTensor()])

trainset=torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=transform)
trainloader=torch.utils.data.DataLoader(trainset,batch_size=32,shuffle=True,num_workers=2)

testset=torchvision.datasets.MNIST(root='./data',train=False,download=True,transform=transform)
testloader=torch.utils.data.DataLoader(trainset,batch_size=32,shuffle=False,num_workers=2)

100.0%


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100.0%


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100.0%


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100.0%

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [3]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel,self).__init__()
        
        #input channel 1 : gray images
        #kernel_size : filter sizes
        #
        self.conv1=nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3)
        
        # Linear layer : (input,output)
        self.d1=nn.Linear(26*26*32,128)
        # 10 is the number of classes
        self.d2=nn.Linear(128,10)
        
    def forward(self,x):
        #32x1x248x28 : 32 is batch size
        x=self.conv1(x)
        x=F.relu(x)
        
        #flatten 32 x (32x26x26)
        x=x.flatten(start_dim=1)
        
        # 32x26x26) ==> 322x128
        x=self.d1(x)
        x=F.relu(x)
        
        #32x10
        logits=self.d2(x)
        out=F.softmax(logits,dim=1)
        return out
        

In [4]:

learning_rate = 0.001
num_epochs = 5

device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model= MyModel()
model=model.to(device)
criterion= nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [5]:
for epoch in range(num_epochs):
    train_running_loss=0.0
    train_acc=0.0
    
    for i, (images,labels)  in enumerate(trainloader):
        images=images.to(device)
        labels=labels.to(device)
        
        logits = model(images)
        loss=criterion(logits,labels)
        
        #if we don't write this: pytorch accumulate gradiants
        optimizer.zero_grad()
        loss.backward() # computes gradiants
        optimizer.step()
        
        train_running_loss += loss.detach().item()
        train_acc += (torch.argmax(logits,1).flatten()==labels).type(torch.float).mean().item()
    
    print('Epoch: %d , Loss: %.4f , Train ACC: %.2f'\
          %(epoch,train_running_loss/i,train_acc/i))

Epoch: 0 , Loss: 1.6392 , Train ACC: 0.83
Epoch: 1 , Loss: 1.4950 , Train ACC: 0.97
Epoch: 2 , Loss: 1.4829 , Train ACC: 0.98
Epoch: 3 , Loss: 1.4779 , Train ACC: 0.99
Epoch: 4 , Loss: 1.4742 , Train ACC: 0.99


In [9]:
test_acc= 0.00
for i,(images,labels) in enumerate(testloader,0):
    images=images.to(device)
    labels=labels.to(device)
    outputs=model(images)
    test_acc += (torch.argmax(outputs,1).flatten()==labels).type(torch.float).mean().item()
    
    preds= torch.argmax(outputs,1).flatten()
    
    print('labels',labels)
    print('pred',preds)
    print('-'*100)
    
print("Test ACC: %0.2f"%(test_acc/i))

labels tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8], device='cuda:0')
pred tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9, 3, 9, 8, 5, 9, 3, 3, 0, 7, 4, 9, 8,
        0, 9, 4, 1, 4, 4, 6, 0], device='cuda:0')
pred tensor([6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9, 3, 9, 8, 5, 9, 3, 3, 0, 7, 4, 9, 8,
        0, 9, 4, 1, 4, 4, 6, 0], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([4, 5, 6, 1, 0, 0, 1, 7, 1, 6, 3, 0, 2, 1, 1, 7, 9, 0, 2, 6, 7, 8, 3, 9,
        0, 4, 6, 7, 4, 6, 8, 0], device='cuda:0')
pred tensor([4, 5, 6, 1, 0, 0, 1, 7, 1, 6, 3, 0, 2, 1, 1, 7, 0, 0, 2, 6, 7, 8, 3, 9,
        0, 4, 6, 7, 4, 6, 8, 0],

pred tensor([9, 7, 2, 8, 1, 9, 5, 8, 8, 1, 9, 8, 3, 1, 6, 5, 7, 4, 2, 2, 0, 3, 0, 1,
        1, 1, 7, 9, 1, 1, 8, 5], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([7, 5, 0, 6, 6, 0, 4, 1, 2, 3, 4, 4, 6, 8, 0, 9, 5, 8, 7, 0, 3, 5, 4, 5,
        9, 6, 7, 1, 9, 6, 1, 3], device='cuda:0')
pred tensor([7, 5, 0, 6, 6, 0, 4, 1, 2, 3, 4, 4, 6, 8, 0, 9, 5, 8, 7, 0, 3, 5, 4, 5,
        9, 6, 7, 1, 9, 6, 1, 3], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([8, 3, 9, 1, 2, 7, 7, 7, 0, 2, 3, 1, 1, 4, 2, 5, 6, 0, 9, 6, 2, 8, 9, 2,
        3, 3, 6, 9, 1, 4, 3, 3], device='cuda:0')
pred tensor([2, 3, 9, 1, 2, 7, 7, 0, 0, 2, 3, 1, 1, 4, 2, 5, 6, 8, 9, 6, 2, 8, 9, 2,
        3, 3, 6, 9, 1, 4, 3, 3], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([0, 

labels tensor([5, 0, 0, 1, 0, 2, 1, 3, 3, 4, 7, 5, 3, 6, 1, 7, 6, 8, 3, 9, 0, 0, 7, 1,
        2, 2, 4, 3, 7, 4, 1, 5], device='cuda:0')
pred tensor([5, 0, 0, 1, 0, 2, 1, 3, 3, 4, 7, 5, 3, 6, 1, 7, 6, 8, 3, 9, 0, 0, 7, 1,
        2, 2, 4, 3, 7, 4, 1, 5], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([0, 6, 3, 7, 9, 8, 7, 9, 5, 0, 1, 1, 8, 2, 6, 3, 3, 4, 3, 5, 3, 7, 1, 8,
        1, 9, 6, 8, 9, 9, 7, 5], device='cuda:0')
pred tensor([0, 6, 3, 7, 9, 8, 7, 9, 5, 0, 1, 1, 8, 2, 6, 3, 3, 4, 3, 5, 3, 7, 1, 8,
        1, 9, 6, 8, 9, 9, 7, 5], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([0, 7, 3, 0, 6, 3, 8, 1, 6, 6, 1, 8, 6, 4, 6, 1, 0, 7, 1, 6, 4, 5, 1, 6,
        7, 4, 8, 2, 5, 7, 3, 8], device='cuda:0')
pred tensor([0, 7, 3, 0, 6, 3, 8, 1, 6, 6, 1, 8, 6, 4, 6, 1, 0, 7, 1, 6, 4, 5, 1, 6,
        7, 4, 8, 2, 5, 7, 3, 8],

labels tensor([6, 5, 4, 6, 7, 5, 2, 0, 8, 1, 9, 2, 8, 1, 3, 7, 8, 8, 8, 1, 2, 4, 8, 3,
        0, 2, 7, 8, 9, 2, 2, 7], device='cuda:0')
pred tensor([6, 5, 4, 6, 7, 5, 2, 0, 8, 1, 9, 2, 8, 1, 3, 7, 8, 8, 8, 1, 2, 4, 8, 3,
        0, 2, 7, 8, 9, 2, 2, 7], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([8, 9, 5, 3, 6, 9, 3, 0, 2, 1, 3, 2, 5, 3, 4, 4, 4, 5, 9, 6, 5, 7, 9, 8,
        3, 9, 4, 0, 8, 1, 0, 2], device='cuda:0')
pred tensor([8, 9, 5, 3, 6, 9, 3, 0, 2, 1, 3, 2, 5, 3, 4, 4, 4, 5, 9, 6, 5, 7, 9, 8,
        3, 9, 4, 0, 8, 1, 0, 2], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([2, 3, 9, 4, 5, 5, 7, 6, 2, 7, 7, 8, 1, 0, 7, 3, 5, 4, 4, 5, 3, 6, 4, 7,
        2, 8, 1, 9, 7, 6, 1, 0], device='cuda:0')
pred tensor([2, 3, 9, 4, 5, 5, 7, 6, 2, 7, 7, 8, 1, 0, 7, 3, 5, 4, 4, 5, 3, 6, 4, 7,
        2, 8, 1, 9, 7, 6, 1, 0],

pred tensor([4, 1, 7, 7, 4, 9, 2, 6, 6, 3, 3, 3, 3, 1, 3, 0, 4, 2, 7, 6, 9, 0, 3, 5,
        1, 6, 3, 6, 4, 8, 7, 7], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([3, 3, 8, 5, 9, 8, 3, 3, 4, 0, 7, 1, 6, 2, 0, 3, 8, 7, 4, 8, 3, 9, 2, 0,
        7, 3, 8, 4, 0, 7, 1, 8], device='cuda:0')
pred tensor([3, 3, 8, 5, 9, 8, 3, 3, 4, 0, 7, 1, 6, 2, 0, 3, 8, 7, 4, 8, 3, 9, 2, 0,
        7, 3, 8, 4, 0, 7, 1, 8], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([0, 0, 5, 1, 4, 2, 1, 3, 7, 4, 7, 5, 1, 6, 5, 7, 5, 8, 9, 9, 8, 5, 0, 1,
        5, 9, 6, 9, 0, 7, 4, 1], device='cuda:0')
pred tensor([0, 0, 5, 1, 4, 2, 1, 3, 7, 4, 7, 5, 1, 6, 5, 7, 5, 8, 9, 9, 8, 5, 0, 1,
        5, 9, 6, 9, 0, 7, 4, 1], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([9, 

...

pred tensor([6, 8, 0, 9, 0, 0, 3, 9, 0, 8, 8, 9, 3, 0, 3, 2, 1, 6, 3, 7, 5, 4, 8, 7,
        7, 5, 7, 0, 0, 1, 1, 2], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([2, 3, 8, 4, 6, 5, 3, 6, 4, 7, 6, 8, 1, 9, 9, 0, 2, 1, 8, 2, 0, 3, 5, 4,
        9, 5, 4, 6, 8, 7, 0, 8], device='cuda:0')
pred tensor([2, 3, 8, 4, 6, 5, 3, 6, 4, 7, 6, 8, 1, 9, 9, 0, 2, 1, 8, 2, 0, 3, 5, 4,
        9, 5, 4, 6, 8, 7, 0, 8], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([0, 0, 2, 1, 1, 2, 3, 3, 3, 4, 8, 5, 1, 6, 3, 8, 6, 3, 9, 4, 3, 7, 8, 8,
        0, 6, 0, 3, 1, 4, 0, 0], device='cuda:0')
pred tensor([0, 0, 2, 1, 1, 2, 3, 3, 3, 4, 8, 5, 1, 6, 3, 8, 6, 3, 9, 4, 3, 7, 8, 8,
        0, 6, 0, 3, 1, 4, 0, 0], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([3, 

labels tensor([2, 0, 7, 1, 4, 2, 9, 3, 8, 4, 2, 5, 2, 6, 6, 7, 6, 8, 1, 9, 8, 0, 8, 1,
        1, 2, 5, 3, 6, 4, 4, 5], device='cuda:0')
pred tensor([2, 0, 7, 1, 4, 2, 9, 3, 8, 4, 2, 5, 2, 6, 6, 7, 6, 8, 1, 9, 8, 0, 8, 1,
        1, 2, 5, 3, 6, 4, 4, 5], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([0, 6, 2, 7, 4, 0, 3, 6, 9, 4, 6, 7, 5, 4, 4, 3, 6, 6, 2, 2, 6, 3, 2, 4,
        7, 8, 3, 7, 9, 8, 1, 6], device='cuda:0')
pred tensor([0, 6, 2, 7, 4, 0, 3, 6, 9, 4, 6, 7, 5, 4, 4, 3, 6, 6, 2, 2, 6, 3, 2, 4,
        7, 8, 3, 7, 9, 8, 1, 6], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([4, 9, 1, 8, 9, 3, 5, 2, 8, 2, 4, 4, 1, 8, 2, 5, 6, 6, 9, 5, 8, 0, 6, 2,
        5, 0, 2, 1, 5, 1, 1, 2], device='cuda:0')
pred tensor([4, 9, 1, 8, 9, 3, 5, 2, 8, 2, 4, 4, 1, 8, 2, 5, 6, 6, 9, 5, 8, 0, 6, 2,
        5, 0, 2, 1, 5, 1, 1, 2],

----------------------------------------------------------------------------------------------------
labels tensor([9, 2, 9, 3, 7, 4, 6, 5, 7, 6, 9, 7, 4, 8, 4, 5, 2, 2, 8, 8, 1, 1, 9, 7,
        2, 2, 9, 4, 6, 1, 3, 4], device='cuda:0')
pred tensor([9, 2, 9, 3, 7, 4, 6, 5, 7, 6, 9, 7, 4, 8, 4, 5, 2, 2, 8, 8, 1, 1, 9, 7,
        2, 2, 9, 4, 6, 1, 3, 4], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([7, 1, 8, 4, 8, 9, 1, 6, 5, 8, 7, 4, 6, 5, 8, 3, 8, 7, 7, 8, 6, 4, 8, 3,
        6, 3, 5, 5, 6, 6, 6, 7], device='cuda:0')
pred tensor([7, 1, 8, 4, 8, 9, 1, 6, 5, 8, 7, 4, 6, 5, 8, 3, 8, 7, 7, 8, 6, 4, 8, 3,
        6, 3, 5, 5, 6, 6, 6, 7], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([3, 0, 1, 6, 7, 1, 0, 6, 8, 8, 7, 7, 2, 0, 9, 1, 6, 5, 8, 6, 3, 0, 5, 8,
        0, 5, 4, 0, 1, 6, 2, 1], device='cuda:0')
pred tensor([3, 

labels tensor([8, 6, 7, 5, 6, 6, 4, 4, 6, 2, 5, 7, 0, 8, 2, 1, 8, 3, 6, 4, 4, 3, 5, 7,
        6, 2, 0, 5, 1, 0, 3, 1], device='cuda:0')
pred tensor([8, 6, 7, 5, 6, 6, 4, 4, 6, 2, 5, 7, 0, 8, 2, 1, 8, 3, 6, 4, 4, 3, 5, 7,
        6, 2, 0, 5, 1, 0, 3, 1], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([1, 9, 3, 2, 9, 3, 6, 2, 1, 3, 5, 5, 7, 5, 2, 7, 6, 8, 8, 4, 4, 9, 2, 9,
        8, 7, 1, 1, 7, 1, 0, 9], device='cuda:0')
pred tensor([1, 9, 3, 2, 9, 3, 6, 2, 1, 3, 5, 5, 7, 5, 2, 7, 6, 8, 8, 4, 9, 9, 2, 9,
        8, 7, 1, 1, 7, 1, 0, 9], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([0, 0, 1, 7, 2, 8, 2, 3, 3, 8, 9, 6, 4, 3, 9, 8, 3, 0, 0, 9, 8, 6, 5, 2,
        6, 1, 9, 0, 8, 1, 7, 0], device='cuda:0')
pred tensor([0, 0, 1, 7, 2, 8, 2, 3, 3, 8, 9, 6, 4, 3, 9, 8, 3, 0, 0, 9, 8, 6, 5, 2,
        6, 1, 9, 0, 8, 1, 7, 0],

pred tensor([2, 1, 4, 0, 1, 2, 3, 4, 7, 8, 9, 0, 1, 2, 3, 4, 7, 8, 9, 0, 1, 2, 3, 4,
        5, 6, 7, 8, 7, 4, 3, 1], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([8, 6, 1, 9, 2, 4, 0, 9, 9, 3, 7, 7, 9, 1, 8, 4, 7, 5, 8, 5, 3, 2, 2, 0,
        5, 8, 6, 0, 3, 8, 1, 0], device='cuda:0')
pred tensor([8, 6, 1, 9, 2, 4, 0, 9, 9, 3, 7, 7, 9, 1, 8, 4, 7, 5, 8, 5, 3, 2, 2, 0,
        5, 8, 6, 0, 3, 8, 1, 0], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([3, 0, 4, 7, 4, 9, 2, 9, 1, 2, 1, 7, 1, 6, 7, 3, 3, 6, 2, 8, 7, 6, 4, 9,
        8, 8, 2, 9, 6, 5, 5, 9], device='cuda:0')
pred tensor([3, 0, 4, 7, 4, 9, 2, 9, 1, 2, 1, 7, 1, 6, 7, 3, 3, 6, 2, 8, 7, 6, 4, 9,
        8, 8, 2, 9, 6, 5, 5, 9], device='cuda:0')
----------------------------------------------------------------------------------------------------
labels tensor([5, 